<a href="https://colab.research.google.com/github/Adeola-001/Chatbot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Installations and Settings

In [ ]:
%%bash
pip install -qqq -U langchain-huggingface
pip install -qqq -U langchain
pip install -qqq -U langchain-community
pip install -qqq -U faiss-cpu
pip install pypdf
pip install unstructured[html]

#### Load data



In [ ]:
import os
from google.colab import userdata

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import UnstructuredHTMLLoader

#### Setting LLM

In [ ]:
hf_model = "mistralai/Mistral-7B-Instruct-v0.3"

llm = HuggingFaceEndpoint(repo_id = hf_model)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


#### Find data


In [ ]:
!wget -O /content/From_Pole_to_Pole https://www.gutenberg.org/cache/epub/20709/pg20709.txt

--2024-06-18 12:26:42--  https://www.gutenberg.org/cache/epub/20709/pg20709.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 964182 (942K) [text/plain]
Saving to: ‘/content/From_Pole_to_Pole’

/content/From_Pole_ 100%[===================>] 941.58K  1.61MB/s    in 0.6s    

2024-06-18 12:26:43 (1.61 MB/s) - ‘/content/From_Pole_to_Pole’ saved [964182/964182]



In [ ]:
!wget -O /content/The_Italian_Cook_Book https://gutenberg.org/cache/epub/24407/pg24407.txt

--2024-06-18 12:26:43--  https://gutenberg.org/cache/epub/24407/pg24407.txt
Resolving gutenberg.org (gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to gutenberg.org (gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206758 (202K) [text/plain]
Saving to: ‘/content/The_Italian_Cook_Book’

/content/The_Italia 100%[===================>] 201.91K   548KB/s    in 0.4s    

2024-06-18 12:26:44 (548 KB/s) - ‘/content/The_Italian_Cook_Book’ saved [206758/206758]



In [ ]:
!wget -O /content/A_Thousand_Ways_to_Please_a_Husband https://gutenberg.org/cache/epub/42868/pg42868.txt

--2024-06-18 12:26:45--  https://gutenberg.org/cache/epub/42868/pg42868.txt
Resolving gutenberg.org (gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to gutenberg.org (gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 661900 (646K) [text/plain]
Saving to: ‘/content/A_Thousand_Ways_to_Please_a_Husband’

/content/A_Thousand 100%[===================>] 646.39K  1.11MB/s    in 0.6s    

2024-06-18 12:26:46 (1.11 MB/s) - ‘/content/A_Thousand_Ways_to_Please_a_Husband’ saved [661900/661900]



In [ ]:
!wget -O /content/With_the_Worlds_Great_Travellers https://gutenberg.org/cache/epub/43745/pg43745.txt

--2024-06-18 12:26:46--  https://gutenberg.org/cache/epub/43745/pg43745.txt
Resolving gutenberg.org (gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to gutenberg.org (gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548685 (536K) [text/plain]
Saving to: ‘/content/With_the_Worlds_Great_Travellers’

/content/With_the_W 100%[===================>] 535.83K  1.12MB/s    in 0.5s    

2024-06-18 12:26:47 (1.12 MB/s) - ‘/content/With_the_Worlds_Great_Travellers’ saved [548685/548685]



In [ ]:
!wget -O /content/Natural_history_in_Anecdote https://www.gutenberg.org/cache/epub/37959/pg37959-images.html

--2024-06-18 12:26:47--  https://www.gutenberg.org/cache/epub/37959/pg37959-images.html
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1011465 (988K) [text/html]
Saving to: ‘/content/Natural_history_in_Anecdote’

/content/Natural_hi 100%[===================>] 987.76K  1.70MB/s    in 0.6s    

2024-06-18 12:26:48 (1.70 MB/s) - ‘/content/Natural_history_in_Anecdote’ saved [1011465/1011465]



In [ ]:
!wget -O /content/International_Monthly_Magazine_of_Literature_Science_and_Art https://www.gutenberg.org/cache/epub/20955/pg20955-images.html

--2024-06-18 12:26:48--  https://www.gutenberg.org/cache/epub/20955/pg20955-images.html
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 904514 (883K) [text/html]
Saving to: ‘/content/International_Monthly_Magazine_of_Literature_Science_and_Art’

/content/Internatio 100%[===================>] 883.31K  1.52MB/s    in 0.6s    

2024-06-18 12:26:49 (1.52 MB/s) - ‘/content/International_Monthly_Magazine_of_Literature_Science_and_Art’ saved [904514/904514]



In [ ]:
!wget -O /content/Travel_pic https://drive.google.com/file/d/12NU-O4qRGxetIqVbm5bichtiG1hmOeBP/view?usp=sharing

--2024-06-18 12:26:49--  https://drive.google.com/file/d/12NU-O4qRGxetIqVbm5bichtiG1hmOeBP/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 108.177.119.113, 108.177.119.100, 108.177.119.139, ...
Connecting to drive.google.com (drive.google.com)|108.177.119.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/content/Travel_pic’

/content/Travel_pic     [ <=>                ]  88.11K  --.-KB/s    in 0.02s   

2024-06-18 12:26:49 (3.76 MB/s) - ‘/content/Travel_pic’ saved [90221]



#### Load all data

In [ ]:
loader = TextLoader("/content/A_Thousand_Ways_to_Please_a_Husband")
documents = loader.load()
loader1 = TextLoader("/content/From_Pole_to_Pole")
documents1 = loader1.load()
loader2 = TextLoader("/content/The_Italian_Cook_Book")
documents2 = loader2.load()
loader3 = TextLoader("/content/With_the_Worlds_Great_Travellers")
documents3 = loader3.load()
loader4 = UnstructuredHTMLLoader("/content/Natural_history_in_Anecdote")
documents4 = loader4.load()
loader5 = UnstructuredHTMLLoader("/content/International_Monthly_Magazine_of_Literature_Science_and_Art")
documents5 = loader5.load()

In [ ]:
#combine all documents
all_documents = documents + documents1 + documents2 + documents3 + documents4 + documents5

#### Split text

In [ ]:
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=250,chunk_overlap=25)

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n","\n","."]
)

docs = text_splitter.split_documents(all_documents)

##### Testing and understanding embedding

In [ ]:
# Embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
#example of embedding
test_text = "Why do data scientists make great comedians?"
query_result = embeddings.embed_query(test_text)
query_result

[0.0017763564828783274,
 -0.05711093172430992,
 -0.03735116124153137,
 0.035620108246803284,
 0.008943280205130577,
 -0.065639927983284,
 0.0797070786356926,
 0.04645843803882599,
 0.03400925174355507,
 0.0417051762342453,
 -0.03569451719522476,
 -0.036951564252376556,
 -0.0495498850941658,
 0.0060961428098380566,
 -0.06058715283870697,
 0.007087082602083683,
 -0.034692250192165375,
 -0.1500234305858612,
 -0.03960023447871208,
 -0.0932970643043518,
 -0.030714672058820724,
 0.03356964886188507,
 0.11057154089212418,
 -0.06375357508659363,
 0.03727630525827408,
 -0.03252798691391945,
 -0.0744345560669899,
 -0.032248206436634064,
 -0.017849203199148178,
 0.0015820838743820786,
 -0.007907777093350887,
 0.04140791296958923,
 0.0769641175866127,
 0.061364322900772095,
 -0.05584949254989624,
 -0.014074950478971004,
 0.03136933594942093,
 0.12421924620866776,
 -0.0316900871694088,
 0.07236950844526291,
 -0.04126685857772827,
 -0.03985981643199921,
 -0.011754604056477547,
 0.031479816883802414,

#### Creating Vector Database

In [ ]:
vector_db = FAISS.from_documents(docs, embeddings)

In [ ]:
vector_db.save_local("/content/faiss_index")

In [ ]:
# new_db = FAISS.load_local("/content/faiss_index", embeddings)

You can also search your database to see which vectors are close to your input.

In [ ]:
vector_db.similarity_search("Cake")

[Document(page_content='=Small Cakes= (Fourteen cakes)\n\n    1-1/4 C-sugar\n    1/3 C-butter\n    2 C-flour\n    4 t-baking powder\n    1/8 t-salt\n    2/3 C-milk\n    1 t-vanilla\n    1/2 t-lemon extract\n    2 egg-whites\n\nCream the butter, add the sugar slowly and continue creaming. Mix and\nsift the flour, baking powder and salt and add these and the milk,\nvanilla and lemon extracts to the butter and sugar. Mix well and beat\ntwo minutes. Beat the egg-whites till very stiff and fold these very\ncarefully into the cake mixture. When thoroughly mixed, fill the cake\npans (which have been prepared with waxed paper) two-thirds of an inch\ndeep with the mixture.\n\nBake twenty-five minutes in a moderate oven, allow to stand five\nminutes, then slip a knife around the edges and remove the cake\ncarefully from the pan. Turn over, remove the paper and allow the cake\nto cool. Ice on the bottom side. When ready for serving, cut in two-inch\nsquares.\n\n\n=Bettina Icing=\n\n1 egg-white 1 

#### Set the template

In [ ]:
input_template = """Answer the question based only on the following context. Keep your answers short and succinct.

Context to answer question:
{context}

Question to be answered: {question}
Response:"""


prompt = PromptTemplate(template=input_template,
                        input_variables=["context", "question"])

#### RAG - chaining it all together


In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = vector_db.as_retriever(search_kwargs={"k": 2}), # top 2 results only, speed things up
    return_source_documents = True,
    chain_type_kwargs = {"prompt": prompt},
)

In [ ]:
answer = qa_chain.invoke("What is the capital of Greece")

answer

{'query': 'What is the capital of Greece',
 'result': ' Mitylene is not the capital of Greece, it is nearer to Athens than Constantinople and its population is almost wholly Greek, but it is not mentioned as the capital of Greece in the context. The capital of Greece is Athens.',
 'source_documents': [Document(page_content='TEMPLE OF JUPITER OLYMPUS.\n\nANOTHER VIEW OF THE TEMPLE OF JUPITER OLYMPUS.\n\nOf two views of the temple of Jupiter Olympus, Mr. Cook chose that in\r\nwhich the Acropolis is seen in the distance. The three lofty Corinthian\r\ncolumns in the other engraving are diminished to the scale of the arch,\r\nwhile the Acropolis, from its greater complexity of parts, adds,\r\nperhaps, something of a quality in which the subject is rather wanting.\r\n"I am not sure," says Mr. Cook, "that the remains of the temple of\r\nJupiter Olympus are not the most impressive which Athens offers to the\r\neye and heart of the traveller, partly from their abstract grandeur—a\r\ngrandeur de

#### Setting up chain

In [ ]:
hf_model = "mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(repo_id=hf_model)

embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

vector_db = FAISS.load_local("/content/faiss_index", embeddings, allow_dangerous_deserialization=True)

retriever = vector_db.as_retriever(search_kwargs={"k": 2})

memory = ConversationBufferMemory(memory_key = 'chat_history',
                                  return_messages = True,
                                  output_key = 'answer')  # Set output_key to 'answer'

template = """You are a nice chatbot having a conversation with a human. Answer the question based only on the following context and previous conversation. Keep your answers short and succinct.

Previous conversation:
{chat_history}

Context to answer question:
{context}

New human question: {question}
Response:"""

prompt = PromptTemplate(template = template,
                        input_variables = ["context", "question"])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# chain
chain = ConversationalRetrievalChain.from_llm(llm,
                                              retriever = retriever,
                                              memory = memory,
                                              return_source_documents = True,
                                              combine_docs_chain_kwargs = {"prompt": prompt})

#### Explore

In [ ]:
chain.invoke("Give me a cake recipe?")

{'question': 'Give me a cake recipe?',
 'chat_history': [HumanMessage(content='Give me a cake recipe?'),
  AIMessage(content=' Here is a recipe for Small Cakes from the provided context:\n\nIngredients:\n- 1-1/4 C sugar\n- 1/3 C butter\n- 2 C flour\n- 4 t baking powder\n- 1/8 t salt\n- 2/3 C milk\n- 1 t vanilla\n- 1/2 t lemon extract\n- 2 egg-whites\n\nInstructions:\n1. Cream the butter, add the sugar slowly and continue creaming.\n2. Mix and sift the flour, baking powder and salt and add these and the milk, vanilla and lemon extracts to the butter and sugar. Mix well and beat 2 minutes.\n3. Beat the egg-whites till very stiff and fold these very carefully into the cake mixture.\n4. Fill the cake pans (prepared with waxed paper) 2/3 inches deep with the mixture.\n5. Bake 25 minutes in a moderate oven.\n6. Allow to stand 5 minutes, then slip a knife around the edges and remove the cake carefully from the pan.\n7. Turn over, remove the paper and allow the cake to cool.\n8. Ice on the bot

In [ ]:
print(chain.invoke("What is a cake?")['answer'])

 The term "cake" originates from the Old Norse word "kaka", which means "to bake". Therefore, in its original language, a cake is called a "kaka".


In [ ]:
print(chain.invoke("Explain the vanilla cake")['answer'])

  To make a vanilla cake, you can use the following recipe:

Ingredients:
- 2 cups all-purpose flour
- 1 cup sugar
- 1/2 cup butter (or margarine)
- 1 1/2 teaspoons baking powder
- 1/2 teaspoon baking soda
- 1/2 teaspoon salt
- 1 cup milk
- 1 teaspoon vanilla extract
- 2 eggs

Instructions:
1. Preheat the oven to 350°F (180°C). Grease and flour a 9-inch round cake pan.
2. In a large bowl, cream the sugar and butter together.
3. Add the eggs, one at a time, beating well after each addition.
4. In a separate bowl, sift together the flour, baking powder, baking soda, and salt.
5. Add half of the flour mixture to the butter and sugar mixture, then add the milk and vanilla extract. Mix well.
6. Add the remaining flour mixture and mix until just combined.
7. Pour the batter into the prepared cake pan and smooth the top with a spatula.
8. Bake for 30-35 minutes, or until a toothpick inserted into the center of the cake comes out clean.
9. Let the cake cool in the pan for 10 minutes, then remo

## Streamlit

#### Test 1

In [ ]:
!pip install -qqq -U streamlit
!npm install -qqq -U localtunnel

In [ ]:
%%writefile app.py
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import streamlit as st

# llm
hf_model= "mistralai/Mistral-7B-Instruct-v0.3"
#llm = HuggingFaceEndpoint(repo_id=hf_model)
llm = HuggingFaceEndpoint(
    repo_id = hf_model,
    max_new_tokens=300,
    temperature=0.8,
    top_k=50,
    top_p=0.9,
    typical_p=0.92,
    repetition_penalty=1.15)

# embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

# load Vector Database
# allow_dangerous_deserialization is needed.
vector_db = FAISS.load_local("/content/faiss_index", embeddings, allow_dangerous_deserialization=True)

# retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

# memory
@st.cache_resource
def init_memory(_llm):
    return ConversationBufferMemory(
        llm=llm,
        output_key='answer',
        memory_key='chat_history',
        return_messages=True)
memory = init_memory(llm)

# prompt
Response= st.selectbox ("In which tone would you like me to talk to you:", ("Sassy", "Funny", "Formal", "Rude"))

template = """You are a nice chatbot having a conversation with a human.
Answer the question based only on the following context and previous conversation. Keep your answers {Response}.

Previous conversation:
{chat_history}

Context to answer question:
{context}

New human question: {question}
Response:"""

prompt = PromptTemplate(template=template,
                        input_variables=["context", "question"])

# chain
chain = ConversationalRetrievalChain.from_llm(llm,
                                              retriever=retriever,
                                              memory=memory,
                                              return_source_documents=True,
                                              combine_docs_chain_kwargs={"prompt": prompt})


##### streamlit #####

st.title("Welcome to the red[OMG] Chat Room :sunglasses:")

#st.markdown(Travel_pic, unsafe_allow_html=True)




# Initialise chat history
# Chat history saves the previous messages to be displayed
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# React to user input
if prompt := st.chat_input("Talk to me"):

    # Display user message in chat message container
    st.chat_message("user").markdown(prompt)

    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    # Begin spinner before answering question so it's there for the duration
    with st.spinner("Mmmh thinking, ideas flowing..."):

        # send question to chain to get answer
        answer = chain(prompt)

        # extract answer from dictionary returned by chain
        response = answer["answer"]

        # Display chatbot response in chat message container
        with st.chat_message("assistant"):
            st.markdown(answer["answer"])

        # Add assistant response to chat history
        st.session_state.messages.append({"role": "assistant", "content": response})

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

#### Test 2

In [ ]:
%%writefile test_app.py
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import streamlit as st
import base64
# Function to add background image
def add_bg_from_local(Travel_pic):
    with open(Travel_pic, "rb") as image:
        encoded_string = base64.b64encode(image.read())
    st.markdown(
        f"""
        <style>
        .stApp {{
            background-image: url(data:image/{"png"};base64,{encoded_string.decode()});
            background-size: cover;
        }}
        </style>
        """,
        unsafe_allow_html=True
    )
# Set the background image
add_bg_from_local('/content/Travel_pic')
# Title of the app
st.title("Welcome to the Travel and Food Recipe Chatbot")
# Introduction text
st.write("Welcome to the Travel and Food Recipe Chatbot! Let's plan your next trip or meal.")
# Option to choose between travel or food suggestion
choice = st.radio(
    "Would you like to:",
    ("Get a travel itinerary", "Get a meal suggestion")
)
if choice == "Get a travel itinerary":
    # User input for travel destinations
    locations = st.text_input("What are the top 3 locations on your bucket list? (separate by commas)")
    if locations:
        # Split the input into a list
        location_list = [location.strip() for location in locations.split(",")]
        if len(location_list) == 3:
            st.write("You have selected the following locations:")
            for location in location_list:
                st.write("- " + location)
            st.write("Here is a suggested itinerary for your trip to these locations...")
            # Here you would add the logic for generating the itinerary
        else:
            st.warning("Please enter exactly 3 locations separated by commas.")
elif choice == "Get a meal suggestion":
    # User input for items in the fridge
    fridge_items = st.text_input("What do you have in your fridge? (separate items by commas)")
    if fridge_items:
        # Split the input into a list
        fridge_list = [item.strip() for item in fridge_items.split(",")]
        st.write("You have the following items in your fridge:")
        for item in fridge_list:
            st.write("- " + item)
        st.write("Based on these items, here is a suggested recipe...")
        # Here you would add the logic for generating the recipe
# LLM Configuration
hf_model = "mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(
    repo_id=hf_model,
    max_new_tokens=300,
    temperature=0.8,
    top_k=50,
    top_p=0.9,
    typical_p=0.92,
    repetition_penalty=1.15
)
# Embeddings Configuration
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model, cache_folder=embeddings_folder)
# Load Vector Database
vector_db = FAISS.load_local("/content/faiss_index", embeddings, allow_dangerous_deserialization=True)
# Retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 2})
# Memory
@st.cache_resource
def init_memory(_llm):
    return ConversationBufferMemory(
        llm=llm,
        output_key='answer',
        memory_key='chat_history',
        return_messages=True
    )
memory = init_memory(llm)
# Prompt
Response = st.selectbox("In which tone would you like me to talk to you:", ("Sassy", "Funny", "Formal", "Rude"))
template = f"""You are a nice chatbot having a conversation with a human.
Answer the question based only on the following context and previous conversation. Keep your answers {Response}.
Previous conversation:
{{chat_history}}
Context to answer question:
{{context}}
New human question: {{question}}
Response:"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])
# Chain
chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    combine_docs_chain_kwargs={"prompt": prompt}
)
##### Streamlit #####

#st.title("Welcome to the OMG Chat Room :sunglasses:")
# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []
# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])
# React to user input
if prompt := st.chat_input("Talk to me"):
    # Display user message in chat message container
    st.chat_message("user").markdown(prompt)
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})
    # Begin spinner before answering question so it's there for the duration
    with st.spinner("Mmmh thinking, ideas flowing..."):
        # Send question to chain to get answer
        answer = chain(prompt)
        # Extract answer from dictionary returned by chain
        response = answer["answer"]
        # Display chatbot response in chat message container
        with st.chat_message("assistant"):
            st.markdown(answer["answer"])
        # Add assistant response to chat history
        st.session_state.messages.append({"role": "assistant", "content": response})

In [ ]:
!streamlit run test_app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

#### Test 3

In [ ]:
%%writefile test_app2.py
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import streamlit as st
import base64
# Function to add background image
def add_bg_from_local(image_file):
    with open(image_file, "rb") as image:
        encoded_string = base64.b64encode(image.read())
    st.markdown(
        f"""
        <style>
        .stApp {{
            background-image: url(data:image/{"png"};base64,{encoded_string.decode()});
            background-size: cover;
        }}
        </style>
        """,
        unsafe_allow_html=True
    )
# Set the background image
add_bg_from_local('/content/Travel_pic')
# Title of the app
st.title("Travel and Food Recipe Chatbot")
# Introduction text
st.write("Welcome to the Travel and Food Recipe Chatbot! Let's plan your next trip or meal.")
# Option to choose between travel or food suggestion
choice = st.radio(
    "Would you like to:",
    ("Get a travel itinerary, "Get a meal suggestion")
)
if choice == "Get a travel itinerary":
    # User input for travel destinations
    locations = st.text_input("What are the top 3 locations on your bucket list? (separate by commas)")
    if locations:
        # Split the input into a list
        location_list = [location.strip() for location in locations.split(",")]
        if len(location_list) == 3:
            st.write("You have selected the following locations:")
            for location in location_list:
                st.write("- " + location)
            st.write("Here is a suggested itinerary for your trip to these locations...")
            # Here you would add the logic for generating the itinerary
        else:
            st.warning("Please enter exactly 3 locations separated by commas.")
elif choice == "Get a meal suggestion":
    # User input for items in the fridge
    fridge_items = st.text_input("What do you have in your fridge? (separate items by commas)")
    if fridge_items:
        # Split the input into a list
        fridge_list = [item.strip() for item in fridge_items.split(",")]
        st.write("You have the following items in your fridge:")
        for item in fridge_list:
            st.write("- " + item)
        st.write("Based on these items, here is a suggested recipe...")
        # Here you would add the logic for generating the recipe

In [ ]:
!streamlit run test_app2.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

#### Test 4

In [ ]:
%%writefile test_app3.py
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import streamlit as st
import base64

# LLM Configuration
hf_model = "mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(
    repo_id=hf_model,
    max_new_tokens=300,
    temperature=0.8,
    top_k=50,
    top_p=0.9,
    typical_p=0.92,
    repetition_penalty=1.15
)

# Embeddings Configuration
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model, cache_folder=embeddings_folder)

# Load Vector Database
vector_db = FAISS.load_local("/content/faiss_index", embeddings, allow_dangerous_deserialization=True)

# Retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

# Initialize Memory
@st.cache_resource
def init_memory(llm_instance):
    return ConversationBufferMemory(
        llm=llm_instance,
        output_key='answer',
        memory_key='chat_history',
        return_messages=True
    )
memory = init_memory(llm)

# User Selection for Tone and Topic
tone = st.selectbox("In which tone would you like me to talk to you:", ("Sassy", "Funny", "Formal", "Rude"))
topic = st.selectbox("Select a conversation topic:", ("Travel", "Food", "Other"))

# Prompt Configuration
template = f"""
You are a nice chatbot having a conversation with a human.
Tailor your responses towards the topic of {topic}. Answer the question based only on the following context and previous conversation. Keep your answers {tone}.
Previous conversation:
{{chat_history}}
Context to answer question:
{{context}}
New human question: {{question}}
Response:"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# Conversational Chain
chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    combine_docs_chain_kwargs={"prompt": prompt}
)

# Streamlit App

# Set page configuration
st.set_page_config(page_title="Chatbot App", page_icon=":robot_face:")

# Add background image using CSS
background_image = """
<style>
body {
    background-image: url("https://drive.google.com/file/d/12NU-O4qRGxetIqVbm5bichtiG1hmOeBP/view?usp=drive_link");
    background-size: cover;
    background-position: center;
}
</style>
"""

st.markdown(background_image, unsafe_allow_html=True)

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# React to user input
user_input = st.chat_input("Talk to me")
if user_input:
    # Display user message in chat message container
    st.chat_message("user").markdown(user_input)
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": user_input})

    # Begin spinner before answering question
    with st.spinner("Mmmh thinking, ideas flowing..."):
        # Send question to chain to get answer
        answer = chain(user_input)
        # Extract answer from dictionary returned by chain
        response = answer["answer"]

        # Display chatbot response in chat message container
        with st.chat_message("assistant"):
            st.markdown(response)
        # Add assistant response to chat history
        st.session_state.messages.append({"role": "assistant", "content": response})


In [ ]:
!streamlit run test_app3.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

#### FINAL VERSION

In [ ]:
%%writefile app.py
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import streamlit as st
import base64

# Title of the app
st.title("Exploring the Beauties of the World Together 🤩 ✈️ 🍝 🏄‍♀️")

# Introduction text
st.write("Tell me, what are you wondering about my darling :heart:?" )

# Set page configuration
#st.set_page_config(page_title="Chatbot App", page_icon=":robot_face:")

# Add background image using CSS
background_image = """
<style>
body {
    background-image: url("https://drive.google.com/uc?id=12NU-O4qRGxetIqVbm5bichtiG1hmOeBP");
    background-size: cover;
    background-position: center;
}
</style>
"""
st.markdown(background_image, unsafe_allow_html= True)

# LLM Configuration
hf_model = "mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(
    repo_id=hf_model,
    max_new_tokens=512,
    temperature=0.8,
    top_k=50,
    top_p=0.9,
    typical_p=0.92,
    repetition_penalty=1.15
)

# Embeddings Configuration
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model, cache_folder=embeddings_folder)

# Load Vector Database
vector_db = FAISS.load_local("/content/faiss_index", embeddings, allow_dangerous_deserialization=True)

# Retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

# Memory
@st.cache_resource
def init_memory(_llm):
    return ConversationBufferMemory(
        llm=llm,
        output_key='answer',
        memory_key='chat_history',
        return_messages=True
    )
memory = init_memory(llm)

# Prompt for tone and topic of response
Response = st.selectbox("In which tone would you like me to talk to you:", ("Formal", "Sassy and Funny", "Rude"))
Topic = st.selectbox("Which topic would you like to talk about:", ("Food", "Travel", "General Knowledge"))

# Function to generate response using selected tone
def generate_response(prompt, tone):
    template = f"""You are a nice chatbot having a conversation with a human.
Answer the question based only on the following context and previous conversation. Keep your answers in a {tone} tone.
Previous conversation:
{{chat_history}}
Context to answer question:
{{context}}
New human question: {{question}}
Response:"""
    prompt_template = PromptTemplate(template=template, input_variables=["context", "question"])
    chain = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=True,
        combine_docs_chain_kwargs={"prompt": prompt_template}
    )
    return chain(prompt)

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# React to user input
if user_input := st.chat_input("Talk to me"):
    # Display user message in chat message container
    st.chat_message("user").markdown(user_input)
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": user_input})
    # Begin spinner before answering question so it's there for the duration
    with st.spinner("Mmmh thinking, ideas flowing..."):
        # Send question to chain to get answer
        answer = generate_response(user_input, Response)
        # Extract answer from dictionary returned by chain
        response = answer["answer"]
        # Display chatbot response in chat message container
        with st.chat_message("assistant"):
            st.markdown(response)
        # Add assistant response to chat history
        st.session_state.messages.append({"role": "assistant", "content": response})


In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com